# Cyber Threat Detection Project

<img src="https://www.tierpoint.com/wp-content/uploads/2023/06/Navigating-the-Cyber-Threat-Landscape-in-2023_blog-1.png.webp">

In [ ]:
# pip install torch transformers datasets scikit-learn matplotlib evaluate

### Import Libraries

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
import torch
from sklearn.model_selection import train_test_split


### Import Dataset

In [ ]:
import kagglehub

path = kagglehub.dataset_download("ramoliyafenil/text-based-cyber-threat-detection")

print("Path to dataset files:", path)

### Dataset Set to Model

In [ ]:
df = pd.read_csv(path + "/cyber-threat-intelligence_all.csv")
df_processed= pd.read_csv(path + "/Cyber-Threat-Intelligence-Custom-Data_new_processed.csv")

In [ ]:
# only text - label_1 and diagnonis
df_processed = df_processed[['text', 'label_1', 'diagnosis']]
#label_1 rename to label
df_processed.rename(columns={'label_1': 'label'}, inplace=True)

In [ ]:
selected_labels = ['malware', 'attack-pattern', 'threat-actor']
data_processed = df_processed[df_processed['label'].isin(selected_labels)]
data_processed.shape

In [ ]:
df = df.dropna(subset=['label'])

selected_labels = ['malware', 'attack-pattern', 'threat-actor']
data = df[df['label'].isin(selected_labels)]
data = data[['text','label']]
data.shape

In [ ]:

data_without_diagnosis = data[~data['text'].isin(data_processed['text'])]
data_with_diagnosis = data_processed.copy()

def auto_generate_diagnosis(label):
    if "malware" in label:
        return "Malicious Software Detected"
    elif "attack-pattern" in label:
        return "Suspicious Attack Pattern Identified"
    elif "threat-actor" in label:
        return "Potential Threat Actor Detected"
    else:
        return "Unknown Activity Detected"


data_without_diagnosis['diagnosis'] = data_without_diagnosis['label'].apply(auto_generate_diagnosis)

combined_data = pd.concat([data_with_diagnosis, data_without_diagnosis], ignore_index=True)
combined_data.head()

In [ ]:
combined_data.shape

In [ ]:
combined_data['combined_label'] = combined_data.apply(lambda row: f"{row['label']}_{row['diagnosis']}", axis=1)
combined_data.drop(columns=['label', 'diagnosis'], inplace=True)

In [ ]:
combined_data.sample(10)

### Model Settings and Training

In [ ]:
# Gerekli kütüphaneleri içe aktarma
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch


def convert_to_dataset(df):
    return Dataset.from_pandas(df)

train_df, test_df = train_test_split(combined_data, test_size=0.2, random_state=42)


train_dataset = convert_to_dataset(train_df)
test_dataset = convert_to_dataset(test_df)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

label_list = sorted(list(set(train_df['combined_label'].tolist())))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

def tokenize_and_prepare(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    labels = []
    for i, combined_label in enumerate(examples["combined_label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_id = label_to_id.get(combined_label, -100)
        label_ids = [-100 if word_id is None else label_id for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(
    tokenize_and_prepare,
    batched=True,
    remove_columns=train_dataset.column_names  
)

test_dataset = test_dataset.map(
    tokenize_and_prepare,
    batched=True,
    remove_columns=test_dataset.column_names  
)


model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_list)
)

training_args = TrainingArguments(
    output_dir='./results',
    eval_steps=500,  
    logging_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)


trainer.train()


evaluation_results = trainer.evaluate()
print(f"Evaluation results: {evaluation_results}")


model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')



### Load Saved Model

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('./saved_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_model')

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

### Prediction

In [ ]:
def predict_example(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = logits.argmax(dim=-1).squeeze().tolist()

    if isinstance(predictions[0], list):
        predictions = predictions[0]

    predicted_labels = [id_to_label.get(pred, "O") for pred in predictions]

    return predicted_labels

In [ ]:
sample_text = "At the time, we found ChessMaster targeting different sectors from the academe to media and government agencies in Japan.."
predicted_labels = predict_example(sample_text)
print(f"Sample text: {sample_text}")
print(f"Predicted label: {predicted_labels[0]}")